# IBM Data Science Capstone Project
#### *By Xavier Cham Villa*
---
## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

Phoenix is a rapidly evolving city with opportunities for new businesses. Over the last 10 years the city has seen a gradual increase in the population, and development expansion to outer areas of Maricopa County, Phoenix's geographical region. A client is looking to invest and expand a consulting business for teh service industry restaurants, nightclubs, etc... As a business moving into new territory the client has asked for recommendations for ideal office locations and useful information on the service industry in our ideal candidate areas



## Data <a name="data"></a>

In order to satisfy our clients demands our approach will be as follows. 

We will develop a 10km radius for selecting the optimal location of our clients new office. Selecting the location consists of centering around mixture restaurants, nightclubs, etc... Furthermore, we shall select an area with middle to upper middle class demographics in order to match our clients main office. Combining these elements our client should be satisfied with our approach. 

Firstly, we will gather data of the Phoenix Metropolitan area. Census data of the last couple of years will be a great resource to use and is readily available at census.ire.org. 

After gathering all our data we will select three cities that satisfy the clients prerequisites and extract other information from city specific census websites. Using a foursquare account we will zero in on our 10km radius of optimal locals into a 3km concentration zone and deliver the verdict to the client. 

* Centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* Number of restaurants/bars and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Phoenix metropolitan area centers will be obtained using **Google Maps API geocoding**

### Gathering Information

Wikipedia, has a good amount of information on Maricopa County, Phoenix and its surrounding cities.I will do some basic web scraping in order to gather information.


In [2]:
# Web scraping Maricopa County Wiki Page
maricopa = requests.get('https://en.wikipedia.org/wiki/Maricopa_County,_Arizona')
html = BeautifulSoup(maricopa.text,'html.parser')
cityinfo = html.find_all('table')[6]

Now we will define a function that extracts and structures table from HTML data.

In [3]:
# Function for extracting table information from an html
def tableData(table):
    def getrow(tr,tag='td'):
        return [td.get_text(strip=True) for td in tr.find_all(tag)]
    rows = []
    tablerows = table.find_all('tr')
    header = getrow(tablerows[0],'th')
    if header:
        rows.append(header)
        tablerows = tablerows[1:]
    
    for r in tablerows:
        rows.append(getrow(r,'td'))
    return rows

Now I will gather information on settlements surrounding the Marciopa County area. We will clean the data and extract only cities and omit towns,ghost towns, etc.. 

In [4]:
# Getting the table data and clensing 
cityTable = tableData(cityinfo)
temp = pd.DataFrame(cityTable)
header = temp.iloc[0]
cityFrame = temp[1:]
cityFrame.columns = header

# Selecting only cities 
cityFrame = cityFrame[(cityFrame['Municipal type']== 'City')]
cityFrame

,Rank,City/Town/etc.,Population (2010 Census),Population (2017 Estimate),Municipal type,Incorporated
1,1,Phoenix†,"1,445,632","1,626,078",City,1881
2,2,Mesa,"439,041","496,401",City,1878 (founded)
3,3,Chandler,"236,123","253,458",City,1920
4,4,Scottsdale,"217,385","249,950",City,1951
5,5,Glendale,"226,721","246,709",City,1910
7,7,Tempe,"161,719","185,038",City,1894
8,8,Peoria(partially inYavapai County),"154,065","168,181",City,1954
9,9,Surprise,"117,517","134,085",City,1960
10,10,Avondale,"76,238","84,025",City,1946
11,11,Goodyear,"65,275","79,858",City,1946


I will focus on cities with only 150k+ residents and omit any others. The United States Census Bureau keeps detailed information on each of city, this is a great resource which we will use to further screen candidates. I will only focus on city service economy, amount of businesses, and resident income.

In [5]:
newFrame = cityFrame.iloc[0:7]
newFrame = newFrame.drop(['Rank','Incorporated','Population (2010 Census)'],axis=1)
newFrame

,City/Town/etc.,Population (2017 Estimate),Municipal type
1,Phoenix†,"1,626,078",City
2,Mesa,"496,401",City
3,Chandler,"253,458",City
4,Scottsdale,"249,950",City
5,Glendale,"246,709",City
7,Tempe,"185,038",City
8,Peoria(partially inYavapai County),"168,181",City


### Phoenix

In [6]:
# Gets census info on city
phoenix = requests.get('https://www.census.gov/quickfacts/phoenixcityarizona')
phoenixhtml = BeautifulSoup(phoenix.text,'html.parser')

# Scrapes info on Accomodation and Food Service Sales
phoenixeconomy = phoenixhtml.find_all('tbody')[10]
phoenixeconomy = tableData(phoenixeconomy)
phoenixeconomy = phoenixeconomy[3][1]

# Scrapes info on income
phoenixincome = phoenixhtml.find_all('tbody')[12]
phoenixincome = tableData(phoenixincome)
phoenixincome = phoenixincome[1][1]

# Scrapes counts on total businesses
phoenixbusiness = phoenixhtml.find_all('tbody')[13]
phoenixbusiness = tableData(phoenixbusiness)
phoenixbusiness = phoenixbusiness[6][1]

### Mesa

In [7]:
# Gets census info on city
mesa = requests.get('https://www.census.gov/quickfacts/mesacityarizona')
mesahtml = BeautifulSoup(mesa.text,'html.parser')

# Scrapes info on Accomodation and Food Service Sales
mesaeconomy = mesahtml.find_all('tbody')[10]
mesaeconomy = tableData(mesaeconomy)
mesaeconomy = mesaeconomy[3][1]

# Scrapes info on income
mesaincome = mesahtml.find_all('tbody')[12]
mesaincome = tableData(mesaincome)
mesaincome = mesaincome[1][1]

# Scrapes counts on total businesses
mesabusiness = mesahtml.find_all('tbody')[13]
mesabusiness = tableData(mesabusiness)
mesabusiness = mesabusiness[6][1]

### Chandler

In [8]:
# Gets census info on city
chandler = requests.get('https://www.census.gov/quickfacts/chandlercityarizona')
chandlerhtml = BeautifulSoup(chandler.text,'html.parser')

# Scrapes info on Accomodation and Food Service Sales
chandlereconomy = chandlerhtml.find_all('tbody')[10]
chandlereconomy = tableData(chandlereconomy)
chandlereconomy = chandlereconomy[3][1]

# Scrapes info on income
chandlerincome = chandlerhtml.find_all('tbody')[12]
chandlerincome = tableData(chandlerincome)
chandlerincome = chandlerincome[1][1]

# Scrapes counts on total businesses
chandlerbusiness = chandlerhtml.find_all('tbody')[13]
chandlerbusiness = tableData(chandlerbusiness)
chandlerbusiness = chandlerbusiness[6][1]

### Scottsdale

In [9]:
# Gets census info on city
scottsdale = requests.get('https://www.census.gov/quickfacts/scottsdalecityarizona')
scottsdalehtml = BeautifulSoup(scottsdale.text,'html.parser')

# Scrapes info on Accomodation and Food Service Sales
scottsdaleeconomy = scottsdalehtml.find_all('tbody')[10]
scottsdaleeconomy = tableData(scottsdaleeconomy)
scottsdaleeconomy = scottsdaleeconomy[3][1]

# Scrapes info on income
scottsdaleincome = scottsdalehtml.find_all('tbody')[12]
scottsdaleincome = tableData(scottsdaleincome)
scottsdaleincome = scottsdaleincome[1][1]

# Scrapes counts on total businesses
scottsdalebusiness = scottsdalehtml.find_all('tbody')[13]
scottsdalebusiness = tableData(scottsdalebusiness)
scottsdalebusiness = scottsdalebusiness[6][1]

### Glendale

In [10]:
# Gets census info on city
glendale = requests.get('https://www.census.gov/quickfacts/glendalecityarizona')
glendalehtml = BeautifulSoup(glendale.text,'html.parser')

# Scrapes info on Accomodation and Food Service Sales
glendaleeconomy = glendalehtml.find_all('tbody')[10]
glendaleeconomy = tableData(glendaleeconomy)
glendaleeconomy = glendaleeconomy[3][1]

# Scrapes info on income
glendaleincome = glendalehtml.find_all('tbody')[12]
glendaleincome = tableData(glendaleincome)
glendaleincome = glendaleincome[1][1]

# Scrapes counts on total businesses
glendalebusiness = glendalehtml.find_all('tbody')[13]
glendalebusiness = tableData(glendalebusiness)
glendalebusiness = glendalebusiness[6][1]

### Tempe

In [11]:
# Gets census info on city
tempe = requests.get('https://www.census.gov/quickfacts/tempecityarizona')
tempehtml = BeautifulSoup(tempe.text,'html.parser')

# Scrapes info on Accomodation and Food Service Sales
tempeeconomy = tempehtml.find_all('tbody')[10]
tempeeconomy = tableData(tempeeconomy)
tempeeconomy = tempeeconomy[3][1]

# Scrapes info on income
tempeincome = tempehtml.find_all('tbody')[12]
tempeincome = tableData(tempeincome)
tempeincome = tempeincome[1][1]

# Scrapes counts on total businesses
tempebusiness = tempehtml.find_all('tbody')[13]
tempebusiness = tableData(tempebusiness)
tempebusiness = tempebusiness[6][1]

### Peoria

In [12]:
# Gets census info on city
peoria = requests.get('https://www.census.gov/quickfacts/peoriacityarizona')
peoriahtml = BeautifulSoup(peoria.text,'html.parser')

# Scrapes info on Accomodation and Food Service Sales
peoriaeconomy = peoriahtml.find_all('tbody')[10]
peoriaeconomy = tableData(peoriaeconomy)
peoriaeconomy = peoriaeconomy[3][1]

# Scrapes info on income
peoriaincome = peoriahtml.find_all('tbody')[12]
peoriaincome = tableData(peoriaincome)
peoriaincome = peoriaincome[1][1]

# Scrapes counts on total businesses
peoriabusiness = peoriahtml.find_all('tbody')[13]
peoriabusiness = tableData(peoriabusiness)
peoriabusiness = peoriabusiness[6][1]

All city information extracted from the census will be compiled into a data frame for further inspection 

In [13]:
# Defining a dictionary containing census data for the selected citites

economyData = [phoenixeconomy,mesaeconomy,chandlereconomy,scottsdaleeconomy,glendaleeconomy,tempeeconomy,peoriaeconomy]
newFrame['Service Sales (Thousands)'] = economyData

incomeData = [phoenixincome,mesaincome,chandlerincome,scottsdaleincome,glendaleincome,tempeincome,peoriaincome]
newFrame['AVG Income'] = incomeData

businessData = [phoenixbusiness,mesabusiness,chandlerbusiness,scottsdalebusiness,glendalebusiness,tempebusiness,peoriabusiness]
newFrame['Count of Businesses'] = businessData
newFrame

,City/Town/etc.,Population (2017 Estimate),Municipal type,Service Sales (Thousands),AVG Income,Count of Businesses
1,Phoenix†,"1,626,078",City,"3,479,625","$54,765","124,033"
2,Mesa,"496,401",City,"685,343","$54,700","33,113"
3,Chandler,"253,458",City,"559,785","$80,716","20,607"
4,Scottsdale,"249,950",City,"1,370,613","$84,601","36,625"
5,Glendale,"246,709",City,"387,297","$52,314","16,783"
7,Tempe,"185,038",City,"646,111","$54,210","17,037"
8,Peoria(partially inYavapai County),"168,181",City,"281,859","$73,039","12,060"


By analyzing the information we can see that by averaging services sales, income, and number of businesses. The top three towns ideal for our client are Phoenix, Mesa, and Scottsdale. We will use these three as potential towns for developing a new business moving forward. 





## Methodology <a name="methodology"></a>

In this first step I have collected the location from each of the city centers (Phoenix,Mesa,Scottsdale). We have also identified the locations and category of restaurants and bars according to our Foursquare categorization. We will compress these into a "top 10" as a useful guide for our client. 

Second step in our analysis will be calculation and exploration of services business density across different areas of the Phoenix Metropolitan area - we will use heatmaps to identify a few promising areas close to center with high number of businesses in general and focus our attention on those areas.

For our final step we will focus primarily on areas in one of these cities with the most types of restaurants and at least 2 bars in order to expose our client to a wide range of potential clients of their own. We will present a map of the potential location to our client by creating clusters to identify  general central point in the selected area. 

## Analysis <a name="analysis"></a>

First, lets gather the geographical coordinates of each city.

In [14]:
saddress = 'Scottsdale,AZ'
geolocator = Nominatim(user_agent="saz_explorer")
slocation = geolocator.geocode(saddress)
slatitude = slocation.latitude
slongitude = slocation.longitude
print('The geograpical coordinate of Scottsdale are {}, {}.'.format(slatitude, slongitude))

maddress = 'Mesa,AZ'
geolocator = Nominatim(user_agent="maz_explorer")
mlocation = geolocator.geocode(maddress)
mlatitude = mlocation.latitude
mlongitude = mlocation.longitude
print('The geograpical coordinate of Mesa are {}, {}.'.format(mlatitude, mlongitude))

paddress = 'Phoenix,AZ'
geolocator = Nominatim(user_agent="paz_explorer")
plocation = geolocator.geocode(paddress)
platitude = plocation.latitude
plongitude = plocation.longitude
print('The geograpical coordinate of Phoenix are {}, {}.'.format(platitude, plongitude))

The geograpical coordinate of Scottsdale are 33.4942189, -111.9260184.
The geograpical coordinate of Mesa are 33.4151117, -111.8314773.
The geograpical coordinate of Phoenix are 33.4484367, -112.0741417.


Here we lay down a function to find restaurants and bars in each city

In [27]:
def get_foursquare(latitude,longitude,radius):
    search_query = 'Restaurant Bars'
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&query={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION,
        search_query,
        latitude, 
        longitude,
        radius, 
        100)
    return url

# Function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
# Function that extracts top services based on location    
def get_services(url):
    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    
    nearby_venues = pd.json_normalize(venues)
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    nearby_venues =nearby_venues.loc[:, filtered_columns]
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    
    return nearby_venues

Getting the top 100 services for each city in a 10km radius and inserting them into a data frame

In [28]:
radius = 10000
cnames = ['city','name','categories','lat','lng']
df = pd.DataFrame(columns = cnames)

scottsdaleURL = get_foursquare(slatitude,slongitude,radius)
scottsdaleServices = get_services(scottsdaleURL)
scottsdaleServices['city'] = 'Scottsdale'

phoenixURL = get_foursquare(platitude,plongitude,radius)
phoenixServices = get_services(phoenixURL)
phoenixServices['city'] = 'Phoenix'

mesaURL = get_foursquare(mlatitude,mlongitude,radius)
mesaServices = get_services(mesaURL)
mesaServices['city'] = 'Mesa'

Services = df.append([scottsdaleServices,mesaServices,phoenixServices],ignore_index=True)
Services

,city,name,categories,lat,lng
0,Scottsdale,Daily Dose Old Town Bar & Grill,Bar,33.494366,-111.926207
1,Scottsdale,CRAFT 64,Pizza Place,33.493339,-111.931653
2,Scottsdale,AZ88,Bar,33.492735,-111.922986
3,Scottsdale,Bandera Restaurant,American Restaurant,33.492336,-111.926008
4,Scottsdale,Olive & Ivy Restaurant + Marketplace,Mediterranean Restaurant,33.500098,-111.928382
...,...,...,...,...,...
295,Phoenix,Chipotle Mexican Grill,Mexican Restaurant,33.494951,-112.014633
296,Phoenix,SanTan Brewing Company,Restaurant,33.434263,-112.010239
297,Phoenix,The Capital Grille,American Restaurant,33.510125,-112.027096
298,Phoenix,Firehouse Subs,Sandwich Place,33.495409,-112.019863


Here we find the number of restaurants and bars based on the their types and categories based on their respective city

In [17]:
# one hot encoding
phoenix_onehot = pd.get_dummies(Services[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
phoenix_onehot['city'] = Services['city'] 

# move neighborhood column to the first column
fixed_columns = [phoenix_onehot.columns[-1]] + list(phoenix_onehot.columns[:-1])
phoenix_onehot = phoenix_onehot[fixed_columns]

phoenix_onehot

,city,American Restaurant,Arepa Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Breakfast Spot,Burger Joint,...,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Scottsdale,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Scottsdale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Scottsdale,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Scottsdale,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Scottsdale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Phoenix,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
296,Phoenix,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
297,Phoenix,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
298,Phoenix,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Finding the top 10 most common venues in each city and their respective frequency

In [51]:
phoenix_grouped = phoenix_onehot.groupby('city').mean().reset_index()

num_top_venues = 10

for hood in phoenix_grouped['city']:
    print("----"+hood+"----")
    temp = phoenix_grouped[phoenix_grouped['city'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Mesa----
                      venue  freq
0        Mexican Restaurant  0.14
1            Sandwich Place  0.09
2               Pizza Place  0.06
3      Fast Food Restaurant  0.04
4          Asian Restaurant  0.04
5            Breakfast Spot  0.04
6              Burger Joint  0.04
7       American Restaurant  0.03
8         Korean Restaurant  0.03
9  Mediterranean Restaurant  0.03


----Phoenix----
                 venue  freq
0   Mexican Restaurant  0.13
1          Pizza Place  0.12
2       Sandwich Place  0.07
3  American Restaurant  0.06
4         Burger Joint  0.05
5                 Café  0.05
6               Bakery  0.04
7                  Bar  0.04
8       Breakfast Spot  0.04
9           Taco Place  0.04


----Scottsdale----
                     venue  freq
0      American Restaurant  0.10
1              Pizza Place  0.08
2       Mexican Restaurant  0.08
3  New American Restaurant  0.06
4               Steakhouse  0.05
5           Breakfast Spot  0.04
6             Burger Joi

For this section we gather the compiled information above, average, and sort venues to the top 10 most common types based on their respective city

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['city']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['city'] = phoenix_grouped['city']

for ind in np.arange(phoenix_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(phoenix_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Mesa,Mexican Restaurant,Sandwich Place,Pizza Place,Fast Food Restaurant,Burger Joint,Asian Restaurant,Breakfast Spot,Korean Restaurant,American Restaurant,Steakhouse
1,Phoenix,Mexican Restaurant,Pizza Place,Sandwich Place,American Restaurant,Café,Burger Joint,Taco Place,Breakfast Spot,Bar,Bakery
2,Scottsdale,American Restaurant,Pizza Place,Mexican Restaurant,New American Restaurant,Steakhouse,Restaurant,Bar,Breakfast Spot,Burger Joint,Italian Restaurant


Here we can visualize restaurants and bars within a 10km radius from the center of each city (Phoenix,Scottsdale,Mesa) displayed in purple. 

In [21]:
services_map = folium.Map(location=[platitude, plongitude+0.1], zoom_start=11)
heat_df = Services.dropna(axis=0,subset=['lat','lng'])
for index, row in heat_df.iterrows():
    folium.CircleMarker([row['lat'], row['lng']],
                        radius=1,
                        fill_color="#3db7e4", # divvy color
                       ).add_to(services_map)
folium.CircleMarker([platitude+0.02,plongitude],radius=22,color = 'purple').add_to(services_map)
folium.CircleMarker([slatitude,slongitude],radius=18,color = 'purple').add_to(services_map)
folium.CircleMarker([mlatitude,mlongitude],radius=18,color = 'purple').add_to(services_map)
services_map

Here we display a heat map of the most popular locations by area

In [22]:
heatdata = [[row['lat'],row['lng']] for index, row in heat_df.iterrows()]
HeatMap(heatdata,blur=15).add_to(services_map)
services_map

This section serves as a filter. We will calculate the number of bars and types of restaurants within a 3km radius of our city centers, and proceed to move on from here.

In [23]:
def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

def count_bars(x,y,bars,radius=3000):
    count = 0
    for index,row in bars.iterrows():
        barx = row['lat']; bary = row['lng']
        d = calc_xy_distance(x, y, barx, bary)
        if d<=radius:
            if row[1]=='Bar':
                count+=1
    return count

def count_restaurants(restaurants):
    df = pd.unique(restaurants['categories'])
    count = len(df)
    return count

sbcount = count_bars(slatitude,slongitude,scottsdaleServices,3000)
srcount = count_restaurants(scottsdaleServices)

mbcount = count_bars(mlatitude,mlongitude,mesaServices,3000)
mrcount = count_restaurants(mesaServices)

pbcount = count_bars(platitude,plongitude,phoenixServices,3000)
prcount = count_restaurants(phoenixServices)

countFrame = {'Bars Count':[sbcount,mbcount,pbcount],'Restaurants Count':[srcount,mrcount,prcount],'City':['Scottsdale','Mesa','Phoenix']}
countFrame = pd.DataFrame(countFrame)
countFrame

,Bars Count,Restaurants Count,City
0,4,39,Scottsdale
1,2,41,Mesa
2,4,33,Phoenix


From the information above we can derive we have 43 different venue locations for Scottsdale and Mesa areas. However, the Scottsdale are has more bars than restaurants, and the Mesa area vice-versa. We will filter out Phoenix from our potential locations since it provides our client to less exposure to an array of different venue categories, and potential clients. We will now generate a heat map with our concentrated venue areas of Scottsdale and Mesa. Furthermore, we will create clusters of venue locations and create a 3 km radius to deliver our final potential locations.

### Scottsdale

In [31]:
df = pd.DataFrame(scottsdaleServices.mean())
radiusNew = 3000
ssnew = get_foursquare(df[0][0],df[0][1],radiusNew)
ssnew = get_services(ssnew)
ssv_map = folium.Map(location=[df[0][0], df[0][1]], zoom_start=11)
heat_df1 = ssnew.dropna(axis=0,subset=['lat','lng'])
for index, row in heat_df1.iterrows():
    folium.CircleMarker([row['lat'], row['lng']],
                        radius=1,
                        fill_color="#3db7e4", # divvy color
                       ).add_to(ssv_map)
heatdata1 = [[row['lat'],row['lng']] for index, row in heat_df1.iterrows()]
HeatMap(heatdata1,blur=15).add_to(ssv_map)
ssv_map

In [84]:
number_of_clusters = 10
hf1 = ssnew[['lat','lng']]
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(hf1)


cluster_centers = kmeans.cluster_centers_
cc = pd.DataFrame(cluster_centers)
ccnames = ['lat','lng']
cc.columns = ccnames
map_ss = folium.Map(location=[df[0][0], df[0][1]], zoom_start=12)
folium.TileLayer('openstreetmap').add_to(map_ss)
folium.Circle([df[0][0], df[0][1]], radius=3000, color='black', fill=True, fill_opacity=0.1).add_to(map_ss)
for index, row in cc.iterrows():
    folium.Circle([row['lat'], row['lng']], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_ss)
map_ss

### Mesa

In [38]:
df2 = pd.DataFrame(mesaServices.mean())
mesanew = get_foursquare(df2[0][0],df2[0][1],radiusNew)
mesanew = get_services(mesanew)
mesa_map = folium.Map(location=[df2[0][0], df2[0][1]], zoom_start=11)
heat_df2 = mesanew.dropna(axis=0,subset=['lat','lng'])
for index, row in heat_df2.iterrows():
    folium.CircleMarker([row['lat'], row['lng']],
                        radius=1,
                        fill_color="#3db7e4", # divvy color
                       ).add_to(mesa_map)
heatdata2 = [[row['lat'],row['lng']] for index, row in heat_df2.iterrows()]
HeatMap(heatdata2,blur=15).add_to(mesa_map)
mesa_map

In [83]:
number_of_clusters = 10
hf2 = mesanew[['lat','lng']]
kmeans2 = KMeans(n_clusters=number_of_clusters, random_state=0).fit(hf2)


cluster_centers2 = kmeans2.cluster_centers_
ss = pd.DataFrame(cluster_centers2)
ssnames = ['lat','lng']
ss.columns = ssnames
map_ms = folium.Map(location=[df2[0][0], df2[0][1]], zoom_start=12)
folium.TileLayer('openstreetmap').add_to(map_ms)
folium.Circle([df2[0][0], df2[0][1]], radius=3000, color='black', fill=True, fill_opacity=0.1).add_to(map_ms)
for index, row in ss.iterrows():
    folium.Circle([row['lat'], row['lng']], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_ms)
map_ms

Now as a final detail for our client. After crawling through the internet we have found that rent prices are evenly distributed throughout the Phoenix metropolitan area, and have displayed them as follows. 

In [26]:
sqdf = requests.get('https://www.squarefoot.com/az/phoenix/office-space')
sqdf = BeautifulSoup(sqdf.text,'html.parser')
sqdf = tableData(sqdf)
temp = pd.DataFrame(sqdf)
header = temp.iloc[0]
sqftFrame = temp[1:]
sqftFrame.columns = header
sqftFrame

,Building Class,Average Rent,Inventory,Vacancy
1,A,$31/sf,42 msf,13%
2,B,$23/sf,53 msf,18%
3,C,$19/sf,7 msf,6%


In [80]:
finalsort = neighborhoods_venues_sorted.drop([1])
finalsort

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Mesa,Mexican Restaurant,Sandwich Place,Pizza Place,Fast Food Restaurant,Burger Joint,Asian Restaurant,Breakfast Spot,Korean Restaurant,American Restaurant,Steakhouse
2,Scottsdale,American Restaurant,Pizza Place,Mexican Restaurant,New American Restaurant,Steakhouse,Restaurant,Bar,Breakfast Spot,Burger Joint,Italian Restaurant


In [82]:
x = ['Mesa','Scottsdale']
finalFrame = newFrame[newFrame['City/Town/etc.'].isin(x)]
finalFrame

,City/Town/etc.,Population (2017 Estimate),Municipal type,Service Sales (Thousands),AVG Income,Count of Businesses
2,Mesa,"496,401",City,"685,343","$54,700","33,113"
4,Scottsdale,"249,950",City,"1,370,613","$84,601","36,625"


## Results and Discussion <a name="results"></a>

Through our analysis we can see that there is a large number of service industry business scattered through the Phoenix, Mesa, and Scottsdale areas in an initial 10km radius start zone for each. We can see that there is connection between these three areas with pockets which are results of mountainous areas, residential zones, and parks. We can also see we see some "bleeding" into the Tempe area which was filtered out as we narrowed down potential zones. 

As a result we reduced our potential zone to a radius of 3km (which was going to be done in proceeding steps). We then identified the number of bars and types of restaurants based around the proximity to our city candidate zones. In order to expose our client to the most amount businesses as possible broke down our list to two potential candidate. Phoenix was dropped as a result of few potential venues based on geolocation and census information was unclear if amount totals for information was accounting for the Phoenix Metropolitan Area or the city itself. 

The result of this is our final locations of the cities of Mesa and Scottsdale. Both have a total 43 distinct venue categories in the service industry which provide our client with some good exposure. We have also supplemented average rent, square footage, service sale estimates, and count of business along with some additional useful information. 

Mesa's potential location zones for our client lie **between East University Drive, East Baseline Road, and between South Dobson Road, South Stapley Drive.**

Scottsdale potential location zones for our client lie **between East McDonald Dr, East McDowell Road, and between North Hayden Road, North 48th Street.**

## Conclusion <a name="conclusion"></a>

The purpose of this research was to provide our client with potential locations for their new business in the Phoenix Metropolitan area. In order to zero in on potential locations some basic filters such as the number of businesses, average income, and services sales were taking into account as filters for each potential city. By calculating the amount of restaurants and bars using Foursquare data we also identified 'hot zones' within the selected cities. Furthermore, additional filters were used in order to deliver our final verdict which will generate our client the optimum locations in which to scout office spaces. Clustering of the data within the selected radius provides our client with even further information on zones of interest.

The final decision on optimal office location will be decided by our client. This will be based on their personal preference and characteristics such as attractiveness, noise pollution, economic dynamics, availability and price. The information provided should satisfy our clients requirements and interests. 